# LLM GAME - Storyteller

본 게임은 스팀 게임 **Storyteller**에서 영감을 받아 제작되었습니다.

플레이어가 시나리오 테마와 배경을 입력하면, LLM이 이를 기반으로 결말과 시나리오에 필요한 인물, 배경 매개체, 최대 컷 수를 생성합니다. 이후 플레이어는 선택한 인물, 매개체, 장소를 활용해 컷별 시나리오 뼈대와 장면을 완성합니다.

플레이어는 최대 컷 수 이내에서 카드와 요소를 조합하며 결말에 다가가는 시나리오를 설계하고, LLM은 플레이어의 입력을 창의적으로 해석해 무한 변주 가능한 스토리를 제공합니다.

In [44]:
import sys
from openai import OpenAI
import json

# from google.colab import userdata
# GPT_API_KEY = userdata.get('GPT_API_KEY')
# client = OpenAI(api_key=GPT_API_KEY)

from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

In [45]:
# 테마, 배경 입력
def theme_and_background():
    # theme = input("테마를 입력하세요: ")
    # background = input("배경을 입력하세요: ")
    
    # 테스트용 하드 코딩
    theme, background = '복수', '중세'

    return theme, background

In [46]:
# 스토리의 구조 생성
def story_structure():
    content = f'''
당신은 창의적이고 논리적인 판타지 스토리 시나리오 전문가입니다.

요청 사항:
1. 입력된 테마와 배경을 바탕으로 결말을 창의적으로 만들어주세요.
단, 간결하게 15자 이내로 간결하게 작성하세요.
2. 결말을 표현하기 위해 필요한 인물, 배경, 매개체를 추천해주세요.
단, 배경은 최소 max_cuts - 2 이상으로 만드세요.
- characters: 스토리에 등장할 캐릭터  
- settings: 장소나 환경  
- objects: 사건에 중요한 오브젝트나 도구, 예를 들어 독약과 독을 탈 술잔, 함정용 도구 등 반드시 포함
3. 결말과 스토리 흐름을 자연스럽게 표현할 수 있는 최대 컷 수(max_cuts)를 결정해주세요.
4. 출력은 반드시 JSON 형식으로 작성하고, 키는 반드시 지정된 명칭으로 사용하세요.
'''\
'''예시 출력:{
    "ending": "기사가 왕위를 강탈한다",
    "characters": ["기사", "왕", "왕비"],
    "settings": ["연회장", "왕좌의 방"],
    "objects": ["술잔", "독약", "왕관"],
    "max_cuts": 5
}'''

    theme, background = theme_and_background()

    user_input = f"테마: {theme}, 배경: {background}"
    print(user_input)
    if '종료' in user_input:
        sys.exit()

    messages = [{"role": "system", "content": content},
                {"role": "user", "content": user_input}]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        response_format = { "type": "json_object" },
        temperature=0.8,
        max_tokens=512,
        top_p=1
    )
    return json.loads(response.choices[0].message.content)

In [47]:
# 시나리오 템플릿 입력
def scenario_template(json_response):
    template = []
    print('''입력은 숫자로 해주세요. 
    여러개를 쓰고 싶으면 , 으로 구분하세요.
    오브젝트를 쓰고싶지 않으면 0 을 입력하세요.
    만약 컷을 그만 쓰고 싶으면 등장인물에서 0 을 입력해주세요.''')
    for i in range(json_response['max_cuts']):
        print(f'{i+1}컷:')

        info = {}
        for label, key in [('등장인물', 'characters'), ('장소', 'settings'), ('오브젝트', 'objects')]:
            print(f'  {label} (숫자 입력): ', end='')
            values = list(map(int,input().strip().split(',')))

            if 0 in values and label == '등장인물':
                print('입력을 종료합니다.')
                break

            te = 0
            for value in values:
                if not 1 <= value <= len(json_response[key]):
                    te = 1
                    break
                
            if te == 1:
                values = [json_response[key][0]]
                print('입력값이 올바르지 않습니다. 1번 값으로 대체합니다. ', end="")
            else:
                values = [json_response[key][value-1] for value in values]
            print(values)
            info[key] = values

        if 0 in values and label == '등장인물':
            break
        template.append(info)

    print("\n최종 시나리오 입력:")
    for i, cut in enumerate(template, 1):
        print(f"  {i}컷 : {cut}")
    
    return template

In [48]:
# 최종 시나리오 작성
def final_scenario(json_response, scenario_templates):
    content = f'''
당신은 창의적이고 논리적인 판타지 스토리 시나리오 전문가입니다.
플레이어가 입력한 컷별 구성을 바탕으로 스토리를 만들어주세요.

스토리 엔딩:
"{json_response['ending']}"

전체 등장인물: {json_response['characters']}
전체 장소: {json_response['settings']}
전체 오브젝트: {json_response['objects']}

요청 사항:
1. 각 컷은 엔딩과 자연스럽게 연결되도록 시나리오를 작성하세요.
2. 선택된 컷으로 엔딩 도달이 가능하면 기존 엔딩을 사용하세요.
3. 만약 불가능하면 새로운 엔딩을 제안하고, 새로운 엔딩으로 가도록 시나리오를 작성하세요.
    - 새로운 엔딩 작성 시 플레이어에게 재도전을 권유하세요.
4. 출력은 반드시 JSON 형식으로 작성하고, 키는 반드시 지정된 명칭으로 사용하세요.
'''\
'''예시 출력:{
    "status": "success" or "retry",
    "final_scenario": ["1컷: ...", "2컷: ..."],
    "ending": "<최종 엔딩>"
}'''

    user_content = "컷별 구성은 아래와 같습니다.\n"
    for i, cut in enumerate(scenario_templates, 1):
        user_content += f"{i}컷: 등장인물 {cut['characters']}, 장소 {cut['settings']}, 오브젝트 {cut['objects']}\n"


    messages = [{"role": "system", "content": content},
                {"role": "user", "content": user_content}]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        response_format = { "type": "json_object" },
        temperature=0.8,
        max_tokens=2048,
        top_p=1
    )

    return json.loads(response.choices[0].message.content)

In [49]:
json_response = story_structure()
json_response

테마: 복수, 배경: 중세


{'ending': '복수의 칼날이 날카롭다',
 'characters': ['복수심 품은 기사', '배신자', '왕'],
 'settings': ['어두운 숲', '성벽', '왕의 방'],
 'objects': ['칼', '배신의 편지', '보물상자'],
 'max_cuts': 6}

In [50]:
print(
    f'''시나리오 엔딩은 다음과 같습니다:
  "{json_response['ending']}"

등장인물: 
{'\n'.join([f'  {i+1}. {j}' for i, j in enumerate(json_response['characters'])])}
장소:
{'\n'.join([f'  {i+1}. {j}' for i, j in enumerate(json_response['settings'])])}
사용 가능한 오브젝트:
{'\n'.join([f'  {i+1}. {j}' for i, j in enumerate(json_response['objects'])])}

{json_response['max_cuts']}컷 이내로 시나리오 뼈대를 작성해보세요.

한 컷에는 1~2명의 등장인물, 1개의 장소, 0~2개의 오브젝트를 사용하세요.'''
)


시나리오 엔딩은 다음과 같습니다:
  "복수의 칼날이 날카롭다"

등장 인물: 
  1. 복수심 품은 기사
  2. 배신자
  3. 왕
장소:
  1. 어두운 숲
  2. 성벽
  3. 왕의 방
사용 가능한 오브젝트:
  1. 칼
  2. 배신의 편지
  3. 보물상자

6컷 이내로 시나리오 뼈대를 작성해보세요.

한 컷에는 1~2명의 등장인물, 1개의 장소, 0~2개의 오브젝트를 사용하세요.


In [51]:
scenario_templates = scenario_template(json_response)

입력은 숫자로 해주세요. 
    여러개를 쓰고 싶으면 , 으로 구분하세요.
    오브젝트를 쓰고싶지 않으면 0 을 입력하세요.
    만약 컷을 그만 쓰고 싶으면 등장 인물에서 0 을 입력해주세요.
1컷:
  등장 인물 (숫자 입력): ['배신자', '왕']
  장소 (숫자 입력): ['왕의 방']
  오브젝트 (숫자 입력): ['칼', '보물상자']
2컷:
  등장 인물 (숫자 입력): ['복수심 품은 기사']
  장소 (숫자 입력): ['왕의 방']
  오브젝트 (숫자 입력): ['칼', '배신의 편지']
3컷:
  등장 인물 (숫자 입력): ['복수심 품은 기사', '배신자']
  장소 (숫자 입력): ['어두운 숲']
  오브젝트 (숫자 입력): ['보물상자']
4컷:
  등장 인물 (숫자 입력): 입력을 종료합니다.

최종 시나리오 입력:
  1컷 : {'characters': ['배신자', '왕'], 'settings': ['왕의 방'], 'objects': ['칼', '보물상자']}
  2컷 : {'characters': ['복수심 품은 기사'], 'settings': ['왕의 방'], 'objects': ['칼', '배신의 편지']}
  3컷 : {'characters': ['복수심 품은 기사', '배신자'], 'settings': ['어두운 숲'], 'objects': ['보물상자']}


In [52]:
scenario_templates

[{'characters': ['배신자', '왕'], 'settings': ['왕의 방'], 'objects': ['칼', '보물상자']},
 {'characters': ['복수심 품은 기사'],
  'settings': ['왕의 방'],
  'objects': ['칼', '배신의 편지']},
 {'characters': ['복수심 품은 기사', '배신자'],
  'settings': ['어두운 숲'],
  'objects': ['보물상자']}]

In [53]:
final_story = final_scenario(json_response, scenario_templates)
final_story

{'status': 'success',
 'final_scenario': ['1컷: 왕의 방에서 배신자는 왕에게 자신이 숨겨놓은 보물상자를 보여주며 성과의 힘을 이용해 왕국을 지배하겠다는 음모를 밝힌다. 왕은 그를 믿고 칼을 쥔 채로 자신의 명령을 기다리고 있다.',
  '2컷: 복수심 품은 기사는 왕의 방에 들어선다. 그의 손에는 배신자의 음모를 담은 배신의 편지가 들려있다. 그는 왕에게 배신자의 진짜 의도를 알리며, 그를 처치할 것을 결심한다.',
  '3컷: 복수심 품은 기사는 배신자를 어두운 숲에서 마주친다. 배신자는 보물상자를 지키며 기사를 조롱하고, 그를 공격하려 하지만, 기사는 배신자의 칼에 맞서 싸운다. 결국, 기사의 복수의 칼날이 날카로워 배신자를 처치하고, 왕국의 평화를 되찾는다.'],
 'ending': '복수의 칼날이 날카롭다'}

In [54]:
print(f"""
시나리오 최종 결과입니다.
{'\n'.join(final_story['final_scenario'])}
결말: {final_story['ending']}
{'성공적인 스토리 텔링입니다!' if final_story['status'] == 'success' else '재도전 해보세요!'}
""")


시나리오 최종 결과입니다.
1컷: 왕의 방에서 배신자는 왕에게 자신이 숨겨놓은 보물상자를 보여주며 성과의 힘을 이용해 왕국을 지배하겠다는 음모를 밝힌다. 왕은 그를 믿고 칼을 쥔 채로 자신의 명령을 기다리고 있다.
2컷: 복수심 품은 기사는 왕의 방에 들어선다. 그의 손에는 배신자의 음모를 담은 배신의 편지가 들려있다. 그는 왕에게 배신자의 진짜 의도를 알리며, 그를 처치할 것을 결심한다.
3컷: 복수심 품은 기사는 배신자를 어두운 숲에서 마주친다. 배신자는 보물상자를 지키며 기사를 조롱하고, 그를 공격하려 하지만, 기사는 배신자의 칼에 맞서 싸운다. 결국, 기사의 복수의 칼날이 날카로워 배신자를 처치하고, 왕국의 평화를 되찾는다.
결말: 복수의 칼날이 날카롭다
성공적인 스토리 텔링입니다!

